<h1> Importing Data </h1>
    
The NFL data from Kaggle is imported and the different files are merged in this notebook to create a dataframe that is used to solve our problem. pandas is used to execute the necessary actions in order to read the files and merge them.

In [1]:
import pandas as pd

scouting_df = pd.read_csv("PFFScoutingData.csv.zip")
games_df = pd.read_csv("games.csv")
players_df = pd.read_csv("players.csv")
plays_df = pd.read_csv("plays.csv.zip")
tracking2018_df = pd.read_csv("tracking2018.csv.zip")
tracking2019_df = pd.read_csv("tracking2019.csv.zip")
tracking2020_df = pd.read_csv("tracking2020.csv.zip")

We only kept the the columns in each of the files from Kaggle that we thought would play a role in predicting whether a fair catch is called or not. 

In [2]:
#creating new dataframe with only the wanted columns
wanted_scouting_df = scouting_df[['gameId','playId','snapDetail','operationTime','hangTime',
                                   'kickType','kickDirectionIntended','kickDirectionActual',
                                   'returnDirectionIntended','kickDirectionActual','kickoffReturnFormation',
                                   'kickContactType']].copy()

wanted_games_df = games_df[['gameId','homeTeamAbbr','visitorTeamAbbr']].copy()

# do we want anything from players_df? 
#wanted_players_df = 

wanted_plays_df = plays_df[['gameId','playId','quarter','down','yardsToGo','possessionTeam','specialTeamsPlayType',
                            'specialTeamsResult','yardlineSide','yardlineNumber','gameClock','penaltyCodes',
                           'penaltyYards','preSnapHomeScore','preSnapVisitorScore','kickLength','kickReturnYardage',
                            'playResult','absoluteYardlineNumber']].copy()

wanted_tracking2018_df = tracking2018_df[['time','x','y','s','a','dis','o','dir','nflId','gameId','playId','playDirection']].copy()

wanted_tracking2019_df = tracking2019_df[['time','x','y','s','a','dis','o','dir','nflId','gameId','playId','playDirection']].copy()

wanted_tracking2020_df = tracking2020_df[['time','x','y','s','a','dis','o','dir','nflId','gameId','playId','playDirection']].copy()



After we created new dataframes with the necessary columns, we then merged them on the gameId and the playId to create one data frame with all the data for each play of every game.

In [3]:
wanted_tracking_df1 = wanted_tracking2018_df.append(wanted_tracking2019_df)

In [4]:
wanted_tracking_df = wanted_tracking_df1.append(wanted_tracking2020_df)

In [5]:
#merge dataframes on gameID and playID
#have to merge two at a time
wanted_df1 = pd.merge(wanted_scouting_df, wanted_plays_df, on=['gameId','playId'])

wanted_df = pd.merge(wanted_df1, wanted_tracking_df, on=['gameId','playId'])



Below are some simple statistics for what percentage of special teams plays fall into each of the four categories: kickoff, field goal, extra point, and punts. 

In [19]:
plays_df.count()
print("there are a total of 19979 plays")
print("there are 4 types of special teams play types: kickoff, punt, field goal, extra point")
plays_df.groupby('specialTeamsPlayType').count()

there are a total of 19979 plays
there are 4 types of special teams play types: kickoff, punt, field goal, extra point


,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsResult,kickerId,returnerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
specialTeamsPlayType,,,,,,,,,,,,,,,,,,,,,
Extra Point,3488,3488,3488,3488,3488,3488,3488,3488,3475,0,...,65,65,65,3488,3488,4,0,0,3488,3488
Field Goal,2657,2657,2657,2657,2657,2657,2657,2657,2643,2,...,40,40,40,2657,2657,10,2606,0,2657,2657
Kickoff,7843,7843,7843,7843,7843,7843,7843,7843,7843,2858,...,356,356,356,7843,7843,0,7843,2969,7843,7843
Punt,5991,5991,5991,5991,5991,5991,5991,5991,5917,4078,...,641,641,641,5991,5991,31,5878,2355,5991,5991


In [6]:
wanted_df2 = pd.merge(wanted_df, wanted_games_df, on=['gameId'])

KeyboardInterrupt: 

In the cells below, we kept only the rows that were for punts and then dropped the specialTeamsPlayType column since it now only contained the value for punt.

In [12]:
## only keeps punts and gets rid of kickoffs, field goals, extra points, etc
punt_df = wanted_df2[wanted_df2['specialTeamsPlayType'] == 'Punt']

In [15]:
punt_df = punt_df.drop(columns=['specialTeamsPlayType'])

We looked to see what the possible outcomes of a punt were and how many occurrences of each outcome there were. We then also chose to only keep the punts that were possible to catch and make the call of a fair catch or not which included getting rid of punts that went out of bounds or were blocked.

In [16]:
punt_df['specialTeamsResult'].unique()

array(['Return', 'Touchback', 'Fair Catch', 'Downed', 'Muffed',
       'Out of Bounds', 'Non-Special Teams Result', 'Blocked Punt'],
      dtype=object)

In [17]:
punt_df = punt_df[punt_df.specialTeamsResult != 'Out of Bounds']

In [19]:
punt_df = punt_df[punt_df.specialTeamsResult != 'Blocked Punt']

In [20]:
punt_df['specialTeamsResult'].unique()

array(['Return', 'Touchback', 'Fair Catch', 'Downed', 'Muffed',
       'Non-Special Teams Result'], dtype=object)

We then exported the final dataframe as a csv file to use for future work.

In [21]:
punt_df.to_csv('punt_df.csv')